In [8]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
from __future__ import print_function # so print doesn't show brackets

sys.path.append(os.path.join("..","Libraries","QML_lib"))
global paulis_list
import Evo as evo
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

from ModelGeneration import *

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


I have tried to name functions that generate true models starting with 'generate_...' and functions used to expand the tree as "expand_..."

## Ising model - chain (1D) - nearest neighbour - 1 direction

- we can probably assume the direction is 'x' in all cases for simplicity
- we already start with the most complicated parametrization: being the $J_{i,j}$ terms randomly chosen, we are exploring what are known as "random (transverse field) Ising models", that are non exactly solvable
- TODO: in literature frequently $J_{i,j} = J \; \forall \{i,j\}$ and we can choose J ~ 0.5 (after rescaling) improving a lot the learning process!

In [17]:
def generate_nnIsing_1D(num_dimensions, mydir = 'x'):
    
    dir_list = [mydir, mydir]
    # todo: make random choice of x,y,z
    list_terms = []
    for i in range(num_dimensions-1):
        list_terms.append(interaction_ham([i,i+1], dir_list, num_dimensions))
        
    
    finalname = (num_dimensions*"P").join(list_terms)
        
    # I am returning also the pure list for you to better check correspondency
    return [finalname, list_terms]

In [18]:
generate_nnIsing_1D(num_dimensions=4)

['xTxTTiTTTiPPPPiTxTTxTTTiPPPPiTiTTxTTTx',
 ['xTxTTiTTTi', 'iTxTTxTTTi', 'iTiTTxTTTx']]

In [19]:
result_gennnIsing1D = generate_nnIsing_1D(5)
result_gennnIsing1D

['xTxTTiTTTiTTTTiPPPPPiTxTTxTTTiTTTTiPPPPPiTiTTxTTTxTTTTiPPPPPiTiTTiTTTxTTTTx',
 ['xTxTTiTTTiTTTTi', 'iTxTTxTTTiTTTTi', 'iTiTTxTTTxTTTTi', 'iTiTTiTTTxTTTTx']]

the expansion for this simpler model generates the new layer from the latest ("name_toexpand"), without knowing the direction of the new interaction term (genereates all ...jTj for $j \in paulis$).

stress-testing by expanding in bigger branches?
- involving e.g. also "something_xTx + something_yTy" (in other words, the tree explores XY or XXZ models?)
- relaxing the directionality of magnetic interactions even? i.e. add "something_xTy" ... 

In [20]:
def expand_nnIsing_1D(name_toexpand, paulis=['x', 'y', 'z']):
    # AAG note: expanding with identity here will be of no use, so I did not use it - hope that makes sense
    
    newdim = 1
    result = 1
    while name_toexpand.count(newdim*"T") > 0:
        newdim = newdim+1
    ### AAG note: when embedding the database, you might want to catch the newdim from the DB entry
    ### e.g. if you can add a default entry DB = None, then 
#     if DB is None: 
#         ... "as above" 
#     else:
#         newdim = DB.loc...
    
    temp_list = name_toexpand.split(newdim*"P")
    
    for i in range(len(temp_list)):
        temp_list[i] = temp_list[i] + newdim*"T" + "i"
    
    newterm = ""   # note AAG: maybe you have a function doing this sort of stuff already that you wanna call here?
    for k in range(newdim-1):
        newterm = newterm + "i" + (k+1)*"T"  
    newterms = [newterm + op + newdim*"T" + op for op in paulis]
    
    # TODO: ok I was struggling to do it in a clever fashion and I ended up desperately sorting out like this.
    # Feel free to improve
    
    newmodels = []
    for newterm in newterms:
        new_list = []
        new_list.extend(temp_list)
        new_list.append(newterm)
#         print(new_list)
        newmodels.append( (newdim*"P").join(new_list) ) 

    return [newmodels, newdim]

In [21]:
result_expnnIsing1D = expand_nnIsing_1D(result_gennnIsing1D[0], paulis=['x'])
result_expnnIsing1D

[['xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTxTTTTTx'],
 5]

## Ising model - chain (1D) - nearest neighbour - "transverse"

- called 'x' the standard interaction direction and 'z' the transverse field direction in all cases for simplicity
- TODO also here we can simplify $J_{i,j} = J \; \forall \{i,j\}$ 
- in theoretical literature we have always $B_i = B \forall i$
 AND the terms with B can be simplified into a single Hamiltonian term, as it then depends upon a single parameter!)
 
The case with constant parameters is exactly solvable for 1D

In [173]:
def generate_nnIsing_trans(num_dimensions, mydir = 'x', trandir = 'z'):
    list_terms = generate_nnIsing_1D(num_dimensions, mydir)[1]
    
    for i in range(num_dimensions):
        list_terms.append(interaction_ham([i], [trandir], num_dimensions))
        
    finalname = (num_dimensions*"P").join(list_terms)
        
    return [finalname, list_terms]

In [174]:
result_gennnIsing_trans = generate_nnIsing_trans(5)
result_gennnIsing_trans

['xTxTTiTTTiTTTTiPPPPPiTxTTxTTTiTTTTiPPPPPiTiTTxTTTxTTTTiPPPPPiTiTTiTTTxTTTTxPPPPPzTiTTiTTTiTTTTiPPPPPiTzTTiTTTiTTTTiPPPPPiTiTTzTTTiTTTTiPPPPPiTiTTiTTTzTTTTiPPPPPiTiTTiTTTiTTTTz',
 ['xTxTTiTTTiTTTTi',
  'iTxTTxTTTiTTTTi',
  'iTiTTxTTTxTTTTi',
  'iTiTTiTTTxTTTTx',
  'zTiTTiTTTiTTTTi',
  'iTzTTiTTTiTTTTi',
  'iTiTTzTTTiTTTTi',
  'iTiTTiTTTzTTTTi',
  'iTiTTiTTTiTTTTz']]

the expansion rule here generates the terms for the new branch as all possible magnetic interactions (xTx, yTy, zTz) and also all possible directions of an external magnetic field, where for relabelling sake we always assume that z would be the trasnverse to an x interaction, x to y, ...

In [175]:
def expand_nnIsing_trans(name_toexpand, paulis=['x', 'y', 'z']):
    
    [newmodels, newdim] = expand_nnIsing_1D(name_toexpand, paulis)
        
    if len(paulis) is 1:
        raise Exception("Cannot build transverse models out of a single direction")
    
    
    newterm = ""   # note AAG: maybe you have a function doing this sort of stuff already that you wanna call here?
    for k in range(newdim):
        newterm = newterm + "i" + (k+1)*"T"  
    newterms = [newterm + op for op in paulis]
    
    newmodels_len = len(newmodels)
    for l in range(newmodels_len):
        lastdir = newmodels[l][-1]
        trandir = paulis.index(lastdir) - 1
        trandir = len(paulis) - 1 if trandir < 0 else trandir
        
        newmodels.append( (newdim*"P").join([newmodels[l], newterms[trandir]])  )   
    
    return [newmodels, newdim]

In [176]:
expand_nnIsing_trans(result_gennnIsing1D[0])

[['xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTxTTTTTx',
  'xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTyTTTTTy',
  'xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTzTTTTTz',
  'xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTxTTTTTxPPPPPiTiTTiTTTiTTTTiTTTTTz',
  'xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTyTTTTTyPPPPPiTiTTiTTTiTTTTiTTTTTx',
  'xTxTTiTTTiTTTTiTTTTTiPPPPPiTxTTxTTTiTTTTiTTTTTiPPPPPiTiTTxTTTxTTTTiTTTTTiPPPPPiTiTTiTTTxTTTTxTTTTTiPPPPPiTiTTiTTTiTTTTzTTTTTzPPPPPiTiTTiTTTiTTTTiTTTTTy'],
 5]

In [1]:
expand_nnIsing_trans(result_gennnIsing_trans[0])

NameError: name 'expand_nnIsing_trans' is not defined

## XY model - chain (1D) - nearest neighbour

$  \mathcal{H} =  \frac{1}{2} \left( \sum_i \frac{1+k}{2} \sigma^x_{i,i+1} + \frac{1-k}{2} \sigma^y_{i,i+1} \right) $

- $k \in [0,1]$, with Ising special case for $k=1$
- also this is exactly solvable (Katsura 1962, McCoy 1971, Jones 1973) - at least in 1D
- $\sigma^x_{i,i+1} = \mathbb{1} \otimes ... \otimes \sigma^x_i \otimes \sigma^x_{i+1} 
\otimes \mathbb{1} \otimes ...  $
- called 'x' & 'y' the standard interaction direction and 'z' the transverse field direction in all cases for simplicity
- TODO also here we can simplify $k_{i,j} = k \; \forall \{i,j\}$ as "k" is known as the anisotropy factor to be constant along the chain

In [188]:
def generate_nnXY(num_dimensions, mydirs = ['x', 'y']):
    
    list_terms = []
    for mydir in mydirs:
        list_terms.extend(generate_nnIsing_1D(num_dimensions, mydir)[1])
        
    finalname = (num_dimensions*"P").join(list_terms)
        
    return [finalname, list_terms]

In [189]:
generate_nnXY(3)

['xTxTTiPPPiTxTTxPPPyTyTTiPPPiTyTTy', ['xTxTTi', 'iTxTTx', 'yTyTTi', 'iTyTTy']]

## XY model - chain (1D) - nearest neighbour, transverse

$ \mathcal{H}  = - \frac{1}{2} \sum_i  \left( \frac{1+k}{2} \sigma^x_{i,i+1} + \frac{1-k}{2} \sigma^y_{i,i+1} 
+ m  B_i \sigma^z_{i} \right) $

- the most general case of $B_i$ depending upon the specific site is almost absent in literature, if $B$ is constant it is exactly treatable in 1D (Katsura 1962)

## XXZ model - chain (1D) - nearest neighbour, transverse

$ \mathcal{H}  = - \frac{1}{2} \sum_i  \left( 
 \sigma^x_{i,i+1} + \sigma^y_{i,i+1}  + \Delta \sigma^z_{i,i+1}
+ m B_i \sigma^z_{i} \right) $

- $\Delta$ is the anisotropy parameter
- solvable with Bethe & Yang-Yang (1966) methods
- the most general case of $B_i$ depending upon the specific site is almost absent in literature

## XYZ model - chain (1D) - nearest neighbour, transverse

$ \mathcal{H}  = - \frac{1}{2} \sum_i  \left( 
 \frac{1+k}{2} \sigma^x_{i,i+1} + \frac{1-k}{2} \sigma^y_{i,i+1}  + \Delta \sigma^z_{i,i+1}
+ m B_i \sigma^z_{i} \right) $

- $\Delta$ is the anisotropy parameter
- special instances treated exactly Baxter (1973) by reducing to XXZ. Limiting cases with $\Delta \rightarrow 0$ yet pose interesting considerations (Jones 1974)
- the most general case of $B_i$ depending upon the specific site is almost absent in literature